In [180]:
import PIL.Image as Image
import glob
import bs4
import numpy as np
import requests
import pandas as pd
from geopy.geocoders import Nominatim
import time



API_KEY=""
API_IP = "127.0.0.1:8080"
POINTS = 90
kmls = glob.glob("images_located/*.kml")
kml_file = open("images_located/captura7.kml", 'r')

segment_image_file = "images_located_segmented/captura7.png"
burnt_image_file = "images_located_burnt/captura7.png"




In [181]:
def get_elevation(lat, long):
    query = 'https://maps.googleapis.com/maps/api/elevation/json?locations=%s, %s&key=%s' % ((lat, long, API_KEY))
    r = requests.get(query).json()
    elevation = pd.io.json.json_normalize(r, 'results')['elevation'].values[0]
    return elevation




In [182]:
clean_kml()



> Answer received by LiquidGalaxy [{"current":{"id":1,"name":"initKmlMaster.kml","path":"/Users/Marcelpv96/kmlApi/initKmlMaster.kml"}}].


In [183]:
segment_image = Image.open(segment_image_file)
burnt_image = Image.open(burnt_image_file)
page = kml_file.read()
bs = bs4.BeautifulSoup(page, "lxml")
coordinates = bs.document.linearring.findAll("coordinates")[0].text.lstrip().lstrip().rstrip().split(' ')[0:4]



In [184]:
def convert_coordinates(coordinates):
    new_coordinates = []
    coor_0 = coordinates[0].split(',')[:2]
    coor_1 = coordinates[1].split(',')[:2]
    coor_2 = coordinates[2].split(',')[:2]
    coor_3 = coordinates[3].split(',')[:2]
    new_coordinates += [coordinates[0]]
    new_coordinates += ["%s,%s,0" % ((coor_0[0], coor_1[1])) ]
    new_coordinates += ["%s,%s,0" % ((coor_3[0], coor_1[1])) ]
    new_coordinates += ["%s,%s,0" % ((coor_3[0], coor_0[1])) ]
    return new_coordinates

In [185]:
coordinates = convert_coordinates(coordinates)

In [186]:
baseLon = float(coordinates[0].split(',')[0])
baseLat = float(coordinates[0].split(',')[1])
lon_2, lat_2 = list(map(float, coordinates[2].split(',')[:2]))
deltaLon = (lon_2 - baseLon)/segment_image.size[0]
deltaLat = (lat_2 - baseLat)/segment_image.size[1]

In [187]:
def check_all_positions(array, i,j):
    return not np.array_equal(np.array((0,0,0)), segment_image_arr[i][j]) and\
                (np.array_equal(np.array((0,0,0)), segment_image_arr[i][j+1]) or\
                 np.array_equal(np.array((0,0,0)), segment_image_arr[i+1][j]) or\
                 np.array_equal(np.array((0,0,0)), segment_image_arr[i+1][j+1]) or\
                 np.array_equal(np.array((0,0,0)), segment_image_arr[i][j-1]) or\
                 np.array_equal(np.array((0,0,0)), segment_image_arr[i-1][j]) or\
                 np.array_equal(np.array((0,0,0)), segment_image_arr[i-1][j-1]))

In [188]:
coordinates_contour = []
segment_image_arr = np.array(segment_image)
burnt_image_arr = np.array(burnt_image)
new_image = segment_image_arr
for i in range(0, segment_image_arr.shape[0]):
    for j in range(0, segment_image_arr.shape[1]):
        if check_all_positions(segment_image_arr, i, j):
            coordinates_contour += [(i, j)]
            burnt_image_arr[i][j] = np.array((255,0,255))

In [189]:
def gelocation_pix(coor):
    lon = deltaLon*coor[1] + baseLon
    lat = deltaLat*coor[0] + baseLat
    return (lon, lat)
geolocated_contour = [gelocation_pix(coor) for coor in coordinates_contour]
geolocated_contour = sorted(geolocated_contour, key = lambda x: x[1])

In [190]:
send_image(burnt_image_file, coordinates[1], coordinates[2], coordinates[3], coordinates[0])

> Send image kml: [images_located_burnt/captura7.png] 
images_located_burnt/captura7.png
{'id': 'captura7.png', 'name': 'captura7.png', 'fCorner': '0.8328270441089125,42.05767969193607,0', 'sCorner': '0.8535321386847095,42.05767969193607,0', 'tCorner': '0.8535321386847095,42.06940813799172,0', 'ftCorner': '0.8328270441089125,42.06940813799172,0'}
> Answer received by LiquidGalaxy [{"message":"Done"}].
> Answer received by LiquidGalaxy [200]. 


In [191]:
lon = geolocated_contour[0][0]
lat = geolocated_contour[0][1]
send_placemark(lon, lat, 'Start')
lon = geolocated_contour[-1][0]
lat = geolocated_contour[-1][1]
send_placemark(lon, lat, 'Finish')




> Sending placemark 
 > Answer received by LiquidGalaxy [{"message":true}].
> Sending placemark 
 > Answer received by LiquidGalaxy [{"message":true}].


In [ ]:
    def fly_drone(path):
        path_splitted = path.split()
        for i in range(1, len(path.split())):
            lon_0 = float(path_splitted[i-1].split(',')[0])
            lat_0 = float(path_splitted[i-1].split(',')[1])

            lon_i = float(path_splitted[i].split(',')[0])
            lat_i = float(path_splitted[i].split(',')[1])
            alpha_lon = (lon_i - lon_0)/3
            alpha_lat = (lat_i - lat_0)/3
            for k in range(0, 3):
                lon_0 += alpha_lon
                lat_0 += alpha_lat
                id = "%d%d" % ((i,k))
                send_placemark(lon_0, lat_0, '', id, 10, 'http://%s/images/icon.png' % API_IP, 2)
                print(id)
                time.sleep(1)
                delete_placemark(id)
        

fly_drone(path)

        

In [ ]:
clean_kml()

In [11]:
def clean_kml():
    url = "http://%s/kml/manage/clean" % API_IP
    response = requests.get(url)
    print("> Answer received by LiquidGalaxy [%s]." % response.text)


def delete_placemark(id):
    url = "http://%s/kml/builder/deleteTag/Placemark/%s" % ((API_IP, id))
    print(url)
    response = requests.delete(url)
    print("> Answer received by LiquidGalaxy [%s]." % response.text)




In [96]:
def real_path(contour):
    real_path = [contour[0]]
    for i  in range(0, len(contour)):
        pixel = contour[i]
        print((abs(pixel[0]-real_path[-1][0])  ,abs(38*deltaLon)), (abs(pixel[1] - real_path[-1][1]) , abs(3*deltaLat)) )
        if abs(pixel[0]-real_path[-1][0])  > abs(38*deltaLon) and abs(pixel[1] - real_path[-1][1]) < abs(3*deltaLat):
            real_path.append(pixel)
            real_path.append((pixel[0], pixel[1]+deltaLat*2))
    return [real_path[i] for i in range(0, len(real_path), 5)]



In [192]:
geolocated_contour

[(0.8384698959710332, 42.06119675417861),
 (0.8384918525930648, 42.06119675417861),
 (0.8385138092150964, 42.06119675417861),
 (0.838535765837128, 42.06119675417861),
 (0.8385577224591596, 42.06119675417861),
 (0.8385796790811912, 42.06119675417861),
 (0.8386016357032228, 42.06119675417861),
 (0.8386235923252544, 42.06119675417861),
 (0.8386455489472859, 42.06119675417861),
 (0.8386675055693176, 42.06119675417861),
 (0.8386894621913492, 42.06119675417861),
 (0.8387114188133807, 42.06119675417861),
 (0.8387333754354124, 42.06119675417861),
 (0.838755332057444, 42.06119675417861),
 (0.8387772886794755, 42.06119675417861),
 (0.8387992453015072, 42.06119675417861),
 (0.8388212019235388, 42.06119675417861),
 (0.8388431585455703, 42.06119675417861),
 (0.838865115167602, 42.06119675417861),
 (0.8388870717896335, 42.06119675417861),
 (0.8389090284116651, 42.06119675417861),
 (0.8389309850336968, 42.06119675417861),
 (0.8389529416557283, 42.06119675417861),
 (0.83897489827776, 42.06119675417861

In [218]:
def new_approach(contour):
    real_path = [contour[0]]
    for i  in range(0, len(contour)):
        pixel = contour[i]
        
        if abs(pixel[0]-real_path[-1][0])  > abs(38*deltaLon) and abs(pixel[1] - real_path[-1][1]) > 1*abs(deltaLat):
            real_path.append(pixel)
            real_path.append((pixel[0], pixel[1]+deltaLat*5))
    return [real_path[i] for i in range(0, len(real_path), 31)]



In [219]:
fin = new_approach(geolocated_contour)

In [220]:
clean_kml()

> Answer received by LiquidGalaxy [{"current":{"id":1,"name":"initKmlMaster.kml","path":"/Users/Marcelpv96/kmlApi/initKmlMaster.kml"}}].


In [221]:
len(fin)

46

In [222]:
path = " ".join([ "".join("%f,%f,20" % ((fin[i][0], fin[i][1]))) for i in range(0, len(fin)) ])
draw_line(path, 'TESTING_PATH', 'path_id')

 > Drawing path
path_id
 > Answer received by LiquidGalaxy [{"message":"done"}].


In [141]:
clean_kml()

> Answer received by LiquidGalaxy [{"current":{"id":1,"name":"initKmlMaster.kml","path":"/Users/Marcelpv96/kmlApi/initKmlMaster.kml"}}].


In [ ]:
for i in range(0, len(new_geolocated_contour)):
    lon = new_geolocated_contour[i][0]
    lat = new_geolocated_contour[i][1]
    send_placemark(lon, lat, i)
    

In [12]:
import requests

def fly_to(lon, lat, range=4000):
    url = "http://%s/kml/flyto/%f/%f/%d" % ((API_IP, lon, lat, range))
    response = requests.get(url)
    print("> Answer received by LiquidGalaxy [%s]." % response.text)

    
def send_image(image, fCorner, sCorner, tCorner, ftCorner):
    url = "http://%s/kml/builder/addPhoto" % API_IP
    print("> Send image kml: [%s] " % image)
    image_file = open(image, 'rb')
    print(image)
    image_name = image.split('/')[-1]
    files = {'img': (image_name, image_file)}
    multipart_form_data = {
        'id': image_name,
        'name': image_name,
        'fCorner': fCorner,
        'sCorner': sCorner,
        'tCorner': tCorner,
        'ftCorner': ftCorner
    }
    print(multipart_form_data)
    fly_to(float(fCorner.split(',')[0]), float(fCorner.split(',')[1]))
    response = requests.post(url, data=multipart_form_data, files=files)
    print("> Answer received by LiquidGalaxy [%s]. "% response.status_code)


def send_placemark(lon, lat, name, id='a', range=0, icon='', scale=2):
    url = "http://%s/kml/builder/addplacemark" % API_IP
    print("> Sending placemark ")
    files = {'img': (None, '')}
    multipart_form_data = {
        'id': id,
        'name': name,
        'longitude': lon,
        'latitude': lat,
        'range': range ,
        'scale': scale,
        'icon': icon,
        'description': ''
    }
    response = requests.post(url, data=multipart_form_data, files = files)
    print(" > Answer received by LiquidGalaxy [%s]." % response.text)
    
    
def draw_line(path, name, id):
    url = "http://%s/kml/builder/drawpath" % API_IP
    print(" > Drawing path")
    files = {'img': (None, '')}
    print(id)
    multipart_form_data = {
        'id': id,
        'name': name,
        'path': path
    }
    response = requests.post(url, data=multipart_form_data, files = files)
    print(" > Answer received by LiquidGalaxy [%s]." % response.text)


    
    

In [ ]:


file_delimitted = glob.glob("../../dronecoria/upload_images/*_delimitted.png")[0]
array_delimitted = np.array(Image.open(file_delimitted))
pos_array = np.array([[(i,j) for j in range(0, array_delimitted.shape[1])] for i in range(0, array_delimitted.shape[0])])
contour = pos_array[np.all( array_delimitted == np.array((255, 0, 255)).reshape(1,1,3), axis=2)]
